In [ ]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import cv2
import time
import datetime
import playsound
import dlib

In [2]:
alarm_on = False
model_path = ''
alarm_path = ''


def eye_aspect_ratio(eye):
    a = distance.euclidean(eye[1], eye[5])
    b = distance.euclidean(eye[2], eye[4])
    c = distance.euclidean(eye[0], eye[3])
    ear = (a+b) / (2.0 * c)
    return ear

In [1]:
def detect():
    thresh = 0.25
    frame_check = 10
    detect = dlib.get_frontal_face_detector()
    predict = dlib.shape_predictor(model_path)

    (lstart, lend) = face_utils.FACIAL_LANDMARKS_68_IDXS['left_eye']
    (rstart, rend) = face_utils.FACIAL_LANDMARKS_68_IDXS['right_eye']

    video = cv2.VideoCapture(0)
    flag = 0
    while True:
        ret, image = video.read()
        if not ret:break
        image = imutils.resize(image, width=640, height=640)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        subjects = detect(gray, 0)
        for subject in subjects:
            shape = predict(gray, subject)
            shape = face_utils.shape_to_np(shape)
            lefteye = shape[lstart:lend]
            righteye = shape[rstart:rend]

            leftear = eye_aspect_ratio(lefteye)
            rightear = eye_aspect_ratio(righteye)

            ear = (leftear + rightear) / 2.0
            lefteyehell = cv2.convexHull(lefteye)
            righteyehell = cv2.convexHull(righteye)

            cv2.drawContours(image, [lefteyehell], -1, (0, 0, 255), 1)
            cv2.drawContours(image, [righteyehell], -1, (0, 0, 255), 1)

            if ear < thresh:
                flag += 1
                if flag >= frame_check:
                    if not alarm_on:
                        alarm_on = True
                        time.sleep(0.001)

                    cv2.putText(image, '********** Drowsy**********', (10, 30), cv2.FONT_ITALIC,
                                1, (0, 0, 255), 1)

                    cv2.putText(image, f'Sleeping Time: {flag-frame_check} sec', (10, 60), cv2.FONT_ITALIC,
                                1, (0, 0, 255), 1)

                    playsound.playsound(alarm_path, True)


            else:
                flag = 0
                alarm_on = False

            cv2.imshow('image', image)
            if cv2.waitKey(1) == ord('o'): break


    cv2.destroyAllWindows()
    video.release()

In [ ]:
detect()